![alt text](https://zewailcity.edu.eg/main/images/logo3.png)

_Prepared by_  [**Muhammad Hamdy AlAref**](mailto:malaref@zewailcity.edu.eg)

# Uninformed Search

Uninformed search algorithms try to solve the problem using only its formulation without any specific knowledge about it. While they are general and, theoretically, can solve any solvable problem, they can be very inefficient!

## Problem Formulation

Problem formulation is the first step in solving it! Basically, problem formulation is rephrasing the problem in a structure that can serve as input to a search algorithms. The following class gives a base for such structure!

In [ ]:
class Problem:
    '''
    Abstract base class for problem formulation.
    It declares the expected methods to be used by a search algorithm.
    All the methods declared are just placeholders that throw errors if not overriden by child "concrete" classes!
    '''

    def __init__(self):
        '''Constructor that initializes the problem. Typically used to setup the initial state and, if applicable, the goal state.'''
        self.init_state = None

    def actions(self, state):
        '''Returns an iterable with the applicable actions to the given state.'''
        raise NotImplementedError

    def result(self, state, action):
        '''Returns the resulting state from applying the given action to the given state.'''
        raise NotImplementedError

    def goal_test(self, state):
        '''Returns whether or not the given state is a goal state.'''
        raise NotImplementedError

    def step_cost(self, state, action):
        '''Returns the step cost of applying the given action to the given state.'''
        raise NotImplementedError

## Node Data Structure

Search algorithms construct trees or graphs of atomic states to try to reach their goal. To keep track of that, some bookkeeping is required. The following class encapsulates such bookkeeping capabilities!

In [ ]:
class Node:
    '''Node data structure for search space bookkeeping.'''

    def __init__(self, state, parent, action, path_cost):
        '''Constructor for the node state with the required parameters.'''
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    @classmethod
    def root(cls, init_state):
        '''Factory method to create the root node.'''
        return cls(init_state, None, None, 0)

    @classmethod
    def child(cls, problem, parent, action):
        '''Factory method to create a child node.'''
        return cls(
            problem.result(parent.state, action),
            parent,
            action,
            parent.path_cost + problem.step_cost(parent.state, action))

def solution(node):
    '''A method to extract the sequence of actions representing the solution from the goal node.'''
    actions = []
    cost = node.path_cost
    while node.parent is not None:
        actions.append(node.action)
        node = node.parent
    actions.reverse()
    return actions, cost

## Example Algorithm: Breadth-First Search (BFS)

Breadth-first search is an uninformed algorithm that searches the state space level-by-level starting from the initial node, i.e., it starts by looking at the root node, then its children, its children's children after that, and so on! Here are example implementations for both its tree version and its graph version.

In [ ]:
from collections import deque

def bfs_tree(problem, verbose=False):
    '''Breadth-first tree search implementation.'''
    if problem.goal_test(problem.init_state): return solution(Node.root(problem.init_state))
    frontier = deque([Node.root(problem.init_state)])
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        node = frontier.pop()
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if problem.goal_test(child.state):
                return solution(child)
            frontier.appendleft(child)

def bfs_graph(problem, verbose=False):
    '''Breadth-first graph search implementation.'''
    if problem.goal_test(problem.init_state): return solution(Node.root(problem.init_state))
    frontier = deque([Node.root(problem.init_state)])
    explored = {problem.init_state}
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        node = frontier.pop()
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored:
                if problem.goal_test(child.state):
                    return solution(child)
                frontier.appendleft(child)
                explored.add(child.state)

## Visualizer

For those of you with sharp eyes, you may have notice a couple of differences from the book pseudo-code. A very obvious one it the `verbose` flag that enables the `Visualizer`! The `Visualizer` is just an auxiliary class that encapsulates some functionalities that can visualize the algorithm's progress. It greatly slows the algorithm down and it is only for illustration purposes! Here is its code, but feel free to skip it altogether!

In [ ]:
from shutil import get_terminal_size
terminal_width, _ = get_terminal_size()

_visualizers = {}

def _default_visualizer(_, state):
    '''Generic visualizer for unknown problems.'''
    print(state)

class Visualizer:
    '''Visualization and printing functionality encapsulation.'''

    def __init__(self, problem):
        '''Constructor with the problem to visualize.'''
        self.problem = problem
        self.counter = 0

    def visualize(self, frontier):
        '''Visualizes the frontier at every step.'''
        self.counter += 1
        print(f'Frontier at step {self.counter}')
        for node in frontier:
            print()
            _visualizers.get(type(self.problem), _default_visualizer)(self.problem, node.state)
        print('-' * terminal_width)

## Example: Sliding Puzzle

The [sliding puzzle](https://en.wikipedia.org/wiki/Sliding_puzzle) is a classic toy problem for search algorithms. It consists of a 2D tile board with one blank tile that the neighboring tiles can slide into to give new configurations of the board. Usually, initial and goal configurations are given and the search algorithm should figure out the steps to go from the initial to the goal.

In [ ]:
class SlidingPuzzle3x3(Problem):
    '''3x3 Sliding Puzzle problem formulation.'''

    def __init__(self, init_state, goal_state):
        assert init_state.count(' ') == 1
        assert goal_state.count(' ') == 1
        self.init_state = tuple(init_state)
        self._goal_state = tuple(goal_state)
        self._action_values = {'up': -3, 'down': +3, 'left': -1, 'right': +1}

    def actions(self, state):
        index = state.index(' ')
        possible_moves = []
        if index // 3 > 0:
            possible_moves.append('up')
        if index // 3 < 2:
            possible_moves.append('down')
        if index % 3 > 0:
            possible_moves.append('left')
        if index % 3 < 2:
            possible_moves.append('right')
        return possible_moves

    def result(self, state, action):
        def swap(t, i, j):
            '''Auxiliary function for swapping two elements in a tuple.'''
            l = list(t)
            l[i], l[j] = l[j], l[i]
            return tuple(l)
        index = state.index(' ')
        return swap(state, index, index + self._action_values[action])

    def goal_test(self, state):
        return state == self._goal_state

    def step_cost(self, state, action):
        return 1

def _sliding_puzzle_3x3_visualizer(problem, state):
    '''Custom visualizer for the 3x3 sliding puzzle problem.'''
    for i in range(0, 9, 3):
        print(' ' + ' '.join(state[i:i + 3]) + ' ')

_visualizers[SlidingPuzzle3x3] = _sliding_puzzle_3x3_visualizer

Let's try solving the sliding puzzle with BFS!

In [ ]:
problem = SlidingPuzzle3x3('12345678 ', '123 56478')

In [ ]:
bfs_graph(problem, verbose=True)

Frontier at step 1

 1 2 3 
 4 5 6 
 7 8   
----------------------------------------------------------------------------------------------------
Frontier at step 2

 1 2 3 
 4 5 6 
 7   8 

 1 2 3 
 4 5   
 7 8 6 
----------------------------------------------------------------------------------------------------
Frontier at step 3

 1 2 3 
 4   5 
 7 8 6 

 1 2   
 4 5 3 
 7 8 6 

 1 2 3 
 4 5 6 
 7   8 
----------------------------------------------------------------------------------------------------
Frontier at step 4

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4   5 
 7 8 6 

 1 2   
 4 5 3 
 7 8 6 
----------------------------------------------------------------------------------------------------
Frontier at step 5

 1   2 
 4 5 3 
 7 8 6 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4   5 
 7 8 6 
----------------------------------------------------------------------------------------------------
Frontier at step 6

 1 2 3 
   4 5 
 7 8 6 

 1 2 

(['left', 'left', 'up'], 3)

In [ ]:
bfs_tree(problem, verbose=True)

Frontier at step 1

 1 2 3 
 4 5 6 
 7 8   
----------------------------------------------------------------------------------------------------
Frontier at step 2

 1 2 3 
 4 5 6 
 7   8 

 1 2 3 
 4 5   
 7 8 6 
----------------------------------------------------------------------------------------------------
Frontier at step 3

 1 2 3 
 4   5 
 7 8 6 

 1 2 3 
 4 5 6 
 7 8   

 1 2   
 4 5 3 
 7 8 6 

 1 2 3 
 4 5 6 
 7   8 
----------------------------------------------------------------------------------------------------
Frontier at step 4

 1 2 3 
 4 5 6 
 7 8   

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4   5 
 7 8 6 

 1 2 3 
 4 5 6 
 7 8   

 1 2   
 4 5 3 
 7 8 6 
----------------------------------------------------------------------------------------------------
Frontier at step 5

 1   2 
 4 5 3 
 7 8 6 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4 5 6 
 7 8   

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4   5 
 7 8 6 

 1 2 3 
 4 5 6 
 7 8   
-

(['left', 'left', 'up'], 3)

What does that mean?

## Requirement

### Part A

Let's re-solve the sliding puzzle problem with other uninformed search algorithms!

You are required to write Python code that implements the following algorithms,

1. Depth-First Search (DFS)
2. Depth-Limited Search (DLS)(**OPTIONAL**)
3. Iterative Deepening Search (IDS)(**OPTIONAL**)

You may think this is a lot, but it is not! If you think about it, by implementing the right one first, the other two can be implemented in terms of it!

After the algorithms are implemented, apply them to the sliding puzzle problem and compare the solutions they find with the one breadth-first search found!

**HINT** For DFS and its variants, recursive implementations are easier and more straightforward!

### Part B

Can you formulate the problem of solving a *2D maze*?

Problem formulation is a *major* part of solving it and it can be *deceivingly* simple! This exercises aims to help you practice it.

Think about the best way to formulate the problem, i.e., proper _state_ and _actions_ representations! Then, implement the `Problem` class interface above.

```python
class Maze(Problem):
    ...
```

Following is a sample for constructing a problem instance (NOT a state representation)!

```python
problem = Maze('''
   #               ##
   #                #
          ####      #
  ##      #     #####
  *#      #      +   
''')
```

where `*` denotes the starting position (initial state), `+` denotes goals and `#` denotes obstacles. The line's length (number of characters) is the maze width (all the lines are of the same width) and the number of lines is the maze hight. Available actions are `up`, `down`, `left` and `right` provided the destination is not an obstacles or out of the maze bounds.

After you are done, you should be able to solve a problem by calling any search algorithm. For example,

```python
bfs_graph(problem)



**Estimated time for this exercise is 1 hour!**

## _Optional_ Generic Sliding Puzzle

This is a generalized formulation for the sliding puzzle problem that enables solving sliding puzzles of any size and labels. This is completely optional.

**DO NOT WASTE TIME UNDERSTANDING IT TILL YOU ARE DONE WITH THE REQUIREMENT!**

In [ ]:
class SlidingPuzzle(Problem):
    '''Generic Sliding Puzzle problem formulation with arbitrary width and height.'''

    blank = ' '

    def __init__(self, init_state, goal_state, width=None, height=None):
        def parse(state):
            '''Auxiliary function for parsing the input into a tuple.'''
            blanks = set(char for char in state if not char.isalnum())
            if len(blanks) is 1:
                blank = blanks.pop()
            elif len(blanks) is 2:
                sep = blanks.pop()
                blank = blanks.pop()
                if state.count(blank) > state.count(sep): sep, blank = blank, sep
                state = state.split(sep)
            else:
                raise ValueError
            state = tuple(state)
            index = state.index(blank)
            return state[:index] + (SlidingPuzzle.blank,) + state[index + 1:]
        self.init_state = parse(init_state)
        self._goal_state = parse(goal_state)
        state_len = len(self.init_state)
        assert state_len is len(self._goal_state)
        self._width = width if width else state_len // height if height else round(state_len ** 0.5)
        self._height = height if height else state_len // width if width else round(state_len ** 0.5)
        assert self._width * self._height is state_len
        self._action_values = {'up': -self._width, 'down': +self._width, 'left': -1, 'right': +1}

    def actions(self, state):
        index = state.index(SlidingPuzzle.blank)
        if index // self._width > 0:
            yield 'up'
        if index // self._width < self._height - 1:
            yield 'down'
        if index % self._width > 0:
            yield 'left'
        if index % self._width < self._width - 1:
            yield 'right'

    def result(self, state, action):
        def swap(t, i, j):
            '''Auxiliary function for swapping two elements in a tuple.'''
            l = list(t)
            l[i], l[j] = l[j], l[i]
            return tuple(l)
        index = state.index(SlidingPuzzle.blank)
        return swap(state, index, index + self._action_values[action])

    def goal_test(self, state):
        return state == self._goal_state

    def step_cost(self, state, action):
        return 1

def _sliding_puzzle_visualizer(problem, state):
    '''Custom visualizer for the sliding puzzle problem.'''
    element_width = max(map(len, state))
    for i in range(0, problem._width * problem._height, problem._width):
        print([i.center(element_width) for i in (state[i:i + problem._width])])

_visualizers[SlidingPuzzle] = _sliding_puzzle_visualizer

<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-11-ad0737cee168>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(blanks) is 1:
<ipython-input-11-ad0737cee168>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif len(blanks) is 2:


In [ ]:
bfs_graph(SlidingPuzzle('1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 #', '1 2 3 4 5 6 7 8 9 10 11 12 # 13 14 15'), verbose=True)

Frontier at step 1

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '12']
['13', '14', '15', '  ']
----------------------------------------------------------------------------------------------------
Frontier at step 2

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '12']
['13', '14', '  ', '15']

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '  ']
['13', '14', '15', '12']
----------------------------------------------------------------------------------------------------
Frontier at step 3

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '  ', '11']
['13', '14', '15', '12']

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '  ']
['9 ', '10', '11', '8 ']
['13', '14', '15', '12']

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '12']
['13', '14', '  ', '15']
----------------------------------------------------------------------------------------------------
Frontier at step 4

['1 ', '2 ', '

(['left', 'left', 'left'], 3)

In [ ]:
from collections import deque

def dfs_tree(problem, verbose=False):
    '''Depth-first tree search implementation.'''
    if problem.goal_test(problem.init_state): return solution(problem.init_state)
    frontier = [Node.root(problem.init_state)]
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        node = frontier.pop()
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if problem.goal_test(child.state):
                return solution(child)
            frontier.append(child)

def dfs_graph(problem, verbose=False):
    '''Depth-first graph search implementation.'''
    if problem.goal_test(problem.init_state): return solution(problem.init_state)
    frontier = deque([Node.root(problem.init_state)])
    explored=set()
    if verbose: visualizer = Visualizer(problem)
    while frontier:
        if verbose: visualizer.visualize(frontier)
        node = frontier.pop()
        if node.state not in explored:
           explored.add(node.state)
           for action in problem.actions(node.state):
                child = Node.child(problem, node, action)
                if problem.goal_test(child.state):
                    return solution(child)
                frontier.append(child)



In [ ]:
dfs_graph(problem, verbose=True)

Frontier at step 1

 1 2 3 
 4 5 6 
 7 8   
----------------------------------------------------------------------------------------------------
Frontier at step 2

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4 5 6 
 7   8 
----------------------------------------------------------------------------------------------------
Frontier at step 3

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5 6 
 7 8   
----------------------------------------------------------------------------------------------------
Frontier at step 4

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 
----------------------------------------------------------------------------------------------------


(['left', 'left', 'up'], 3)

In [ ]:
dfs_tree(problem, verbose=True)

Streaming output truncated to the last 5000 lines.
 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 


KeyboardInterrupt: 

In [ ]:
dfs_graph(SlidingPuzzle('1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 #', '1 2 3 4 5 6 7 8 9 10 11 12 # 13 14 15'), verbose=True)

Frontier at step 1

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '12']
['13', '14', '15', '  ']
----------------------------------------------------------------------------------------------------
Frontier at step 2

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '  ']
['13', '14', '15', '12']

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '12']
['13', '14', '  ', '15']
----------------------------------------------------------------------------------------------------
Frontier at step 3

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '  ']
['13', '14', '15', '12']

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '  ', '12']
['13', '14', '11', '15']

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '12']
['13', '  ', '14', '15']

['1 ', '2 ', '3 ', '4 ']
['5 ', '6 ', '7 ', '8 ']
['9 ', '10', '11', '12']
['13', '14', '15', '  ']
----------------------------------

(['left', 'left', 'left'], 3)



where * denotes the starting position (initial state), + denotes goals and # denotes obstacles. The line's length (number of characters) is the maze width (all the lines are of the same width) and the number of lines is the maze hight. Available actions are up, down, left and right provided the destination is not an obstacles or out of the maze bounds.

#2-D Maze


In [ ]:
from math import ceil
class maze(Problem):
    " 2-D Maze"
    def __init__(self, init_state):
        start=init_state.find("\n")
        end=init_state.find("\n",start+1)
        self.width=end-start
        self.hight=init_state.count("\n")-1
        assert len(init_state)==self.width*self.hight+1
        for i in range(self.hight-1):
          start=init_state.find("\n",start+1)
          end=init_state.find("\n",start+1)
          assert (end-start)==self.width
        self.goal_state= init_state.find("+")
        self.init_state = tuple(init_state)
        self._action_values = {'up': -self.width, 'down': +self.width, 'left': -1, 'right': +1}
    def actions(self, state):
        index = state.index('*')
        possible_moves = []
        if (index // self.width > 0) and (state[index-self.width]!="#"):
            possible_moves.append('up')
        if (index // self.width < self.hight-1) and (state[index+self.width]!="#"):
            possible_moves.append('down')
        if (index % self.width > 1) and (state[index-1]!="#"):
            possible_moves.append('left')
        if (index % self.width < self.width-2) and (state[index+1]!="#"):
            possible_moves.append('right')
        return possible_moves
    def result(self, state, action):
        def swap(t, i, j):
            '''Auxiliary function for swapping two elements in a tuple.'''
            l = list(t)
            l[i], l[j] = l[j], l[i]
            return tuple(l)
        index = state.index("*")
        return swap(state, index, index + self._action_values[action])

    def goal_test(self, state):
        return state.index("*") == self.goal_state

    def step_cost(self, state, action):
        return 1

def maze_visualizer(problem, state):
    '''Custom visualizer for the Maze problem.'''
    print(''.join(state))

_visualizers[maze] = maze_visualizer

In [ ]:
problem =('''
   #                #
   #                #
          ####      #
  ##      #     #####
  *#      #      +
''')

In [ ]:
maze(problem)

In [ ]:
print(bfs_graph(maze(problem)))

(['left', 'up', 'up', 'right', 'right', 'right', 'up', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'down', 'down', 'down', 'right', 'right', 'right'], 23)


In [ ]:
print(dfs_graph(maze(problem)))

(['left', 'left', 'up', 'right', 'up', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'up', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'right', 'down', 'left', 'left', 'left', 'left', 'left', 'down', 'right', 'down', 'right', 'right'], 35)


initial class..but gives errors when changing the dimensions of the problem

In [ ]:
import math
class Maze(Problem):
    '''Generic Sliding Puzzle problem formulation with arbitrary width and height.'''

    blank = ' '

    def __init__(self, init_state):
        '''          * denotes the starting position (initial state),
                     + denotes goals and
                     # denotes obstacles.
        '''
        self.init_state = tuple(init_state)

        self.position = init_state.index('*')
        self.goal = init_state.index('+')
        state_len = len(self.init_state)
        #assert state_len is len(self._goal_state)
        #self._width = width if width else state_len // height if height else round(state_len ** 0.5)
        #self._height = height if height else state_len // width if width else round(state_len ** 0.5)
        #assert self._width * self._height is state_len
        self._shape= math.isqrt(state_len)
        #if self._shape * self._shape < state_len:
        #    print("cannot assert a square layout")
        #else:
          #assert self._shape * self._shape == state_len
        self._action_values = {'up': -self._shape, 'down': +self._shape, 'left': -1, 'right': +1}

    def actions(self, state):
       #index = state.index(SlidingPuzzle.blank) ===>>> self.position
        '''Available actions are up, down, left and right
        provided the destination is not an obstacles
        or
        out of the maze bounds.'''
        index = self.position
        row, col = divmod(index, self._shape)
        if row > 0 and state[index - self._shape] != '#':
           yield 'up'
        if row < self._shape - 1 and state[index + self._shape] != '#':
          yield 'down'
        if col > 0 and state[index - 1] != '#':
           yield 'left'
        if col < self._shape - 1 and state[index + 1] != '#':
           yield 'right'
    def result(self, state, action):
        def swap(t, i, j):
            '''Auxiliary function for swapping two elements in a tuple.'''
            l = list(t)
            l[i]=' '
            l[j]='*'
            return tuple(l)
        #index = state.index(SlidingPuzzle.blank) ===>>> self.position
        self.position=state.index('*')
        return swap(state, self.position, self.position + self._action_values[action])

    def goal_test(self, state):
        return state.index('*') == self.goal

    def step_cost(self, state, action):
        return 1

def maze_visualizer(problem, state):
    '''Custom visualizer for the sliding puzzle problem.'''
    element_width = max(map(len, state))
    for i in range(0, len(state), problem._shape): ############
        print([i.center(element_width) for i in (state[i:i + problem._shape])])
_visualizers[Maze] = maze_visualizer

In [ ]:
problem = Maze('###*### # #+########')
#problem = Maze('# ####  ####      ### #     #####*# # +#   ')
print("State Length:", len(problem.init_state))
print("Calculated Shape:", problem._shape)
print(bfs_graph(problem))


State Length: 20
Calculated Shape: 4
(['down', 'down'], 2)
